<a href="https://colab.research.google.com/github/drivera27/Employee-Churn-Project/blob/main/EmployeeChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Employee Churn Analytics Project

**To-do:**
* Prepare `df` for classification
  * Drop `employee_id`
* Perform quick EDA to determine which metrics to use
  * Are the classes (quit vs. non-quit) even?
* Use a RandomForest classifier
  * Import `scikit-learn` RF
* Add predictions and prediction scores to `df`
* Upload updated `df` to BigQuery

## Setup

### Importing libraries

In [1]:
# import libraries
from google.cloud import bigquery
from google.colab import auth

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

### Retrieving data via BigQuery

In [ ]:
auth.authenticate_user()

# initialize client for bigquery
project_id = 'essential-cairn-479919-a3'
client = bigquery.Client(project=project_id, location='US')

# connect to BigQuery database
dataset_ref = client.dataset('EmployeeData', project=project_id)
dataset = client.get_dataset(dataset_ref)

In [48]:
# retrieve HR table from BigQuery
HR_ref = dataset.table(table_id='HRdata')
HR_table = client.get_table(HR_ref)

# convert to pandas DataFrame
df = client.list_rows(table=HR_table).to_dataframe()

# print first 5 rows and verify
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,Quit_the_Company,promotion_last_5years,Departments,salary,employee_id
0,0.38,0.53,2,157,3,0,1,0,sales,low,EMP00001
1,0.80,0.86,5,262,6,0,1,0,sales,medium,EMP00002
2,0.11,0.88,7,272,4,0,1,0,sales,medium,EMP00003
3,0.72,0.87,5,223,5,0,1,0,sales,low,EMP00004
4,0.37,0.52,2,159,3,0,1,0,sales,low,EMP00005


In [49]:
# retrieve new employees table from BigQuery
new_table_ref = dataset.table('NewEmployees')
new_table = client.get_table(new_table_ref)

# create dataframe for new employees table
df2 = client.list_rows(table=new_table).to_dataframe()

# print first 5 rows and verify
df2.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,Quit_the_Company,promotion_last_5years,Departments,salary,employee_id
0,0.537849,0.122914,2,208,2,0,0,0,IT,high,NE15043
1,0.056211,0.322600,2,229,5,1,0,0,IT,low,NE15086
2,0.555186,0.555949,2,187,3,0,0,0,RandD,medium,NE15035
3,0.605273,0.713086,2,218,3,0,0,0,RandD,high,NE15069
4,0.043437,0.162372,2,175,3,0,0,0,RandD,high,NE15082


## Modeling

### Preparing data for modeling

In [57]:
# drop non-relevant columns (employee ID)
df = df.drop(columns=['employee_id'])
df2 = df2.drop(columns=['employee_id'])

KeyError: "['employee_id'] not found in axis"

In [58]:
# separate independent and dependent (target) features
X = df.drop(columns=['Quit_the_Company'], axis=1)
y = df['Quit_the_Company']

In [75]:
# identify categorical and numerical features
categorical = ['Departments', 'salary']
numerical = X.columns[:-2].tolist()

# transform categorical variables to numerical
# use one-hot encoding ?

In [80]:
df['Departments'].unique()

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)